In [1]:
from df.enhance import enhance, init_df, save_audio
import librosa
import torch
import numpy as np
import soundfile as sf

c:\Users\ujjan\AppData\Local\Programs\Python\Python310\lib\site-packages\df\io.py:9: UserWarning: `torchaudio.backend.common.AudioMetaData` has been moved to `torchaudio.AudioMetaData`. Please update the import path.
  from torchaudio.backend.common import AudioMetaData


In [2]:
def denoizer(noisy,output):
    noisy_audio = noisy
    # Initialize the model and state
    model, df_state, _ = init_df()
    # Load the audio file using librosa
    audio, sr = librosa.load(noisy_audio, sr=df_state.sr()) # Test loading the file
    print(f"Audio loaded successfully with sample rate: {sr}")

    # Convert the NumPy array to a PyTorch Tensor
    audio_tensor = torch.tensor(audio, dtype=torch.float32)
    # Add batch dimension: [1, num_samples]
    audio_tensor = audio_tensor.unsqueeze(0)
    # Enhance the audio
    enhanced = enhance(model, df_state, audio_tensor)
    # Remove batch dimension from the output
    enhanced = enhanced.squeeze(0)
    # Convert the enhanced tensor to a NumPy array
    enhanced_np = enhanced.numpy()
    output_path = output
    sf.write(output_path, enhanced_np, sr)

In [3]:
noize = r"C:\Users\ujjan\OneDrive\Desktop\python_projects\GGWave\noisy_snr0.wav"
output_path = r"C:\Users\ujjan\OneDrive\Desktop\python_projects\GGWave\enhanced.wav"
denoizer(noize,output_path)

2025-03-29 01:23:23 | INFO     | DF | Running on torch 2.6.0+cpu
2025-03-29 01:23:23 | INFO     | DF | Running on host Sundram
2025-03-29 01:23:23 | INFO     | DF | Loading model settings of DeepFilterNet3
2025-03-29 01:23:23 | INFO     | DF | Using DeepFilterNet3 model at C:\Users\ujjan\AppData\Local\DeepFilterNet\DeepFilterNet\Cache\DeepFilterNet3
2025-03-29 01:23:23 | INFO     | DF | Initializing model `deepfilternet3`
2025-03-29 01:23:23 | INFO     | DF | Found checkpoint C:\Users\ujjan\AppData\Local\DeepFilterNet\DeepFilterNet\Cache\DeepFilterNet3\checkpoints\model_120.ckpt.best with epoch 120
2025-03-29 01:23:23 | INFO     | DF | Running on device cpu
2025-03-29 01:23:23 | INFO     | DF | Model loaded
Audio loaded successfully with sample rate: 48000


In [4]:
import ggwave
print(dir(ggwave))

['__builtins__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '__test__', 'decode', 'disableLog', 'enableLog', 'encode', 'free', 'getDefaultParameters', 'init', 're', 'rxToggleProtocol', 'txToggleProtocol']


In [5]:
noise = r"C:\Users\ujjan\OneDrive\Desktop\python_projects\GGWave\output.wav"
output_path = r"C:\Users\ujjan\OneDrive\Desktop\python_projects\GGWave\enhanced.wav"
denoizer(noise,output_path)

2025-03-29 01:23:29 | INFO     | DF | Loading model settings of DeepFilterNet3
2025-03-29 01:23:29 | INFO     | DF | Using DeepFilterNet3 model at C:\Users\ujjan\AppData\Local\DeepFilterNet\DeepFilterNet\Cache\DeepFilterNet3
2025-03-29 01:23:29 | INFO     | DF | Initializing model `deepfilternet3`
2025-03-29 01:23:29 | INFO     | DF | Found checkpoint C:\Users\ujjan\AppData\Local\DeepFilterNet\DeepFilterNet\Cache\DeepFilterNet3\checkpoints\model_120.ckpt.best with epoch 120
2025-03-29 01:23:29 | INFO     | DF | Running on device cpu
2025-03-29 01:23:29 | INFO     | DF | Model loaded
Audio loaded successfully with sample rate: 48000


In [26]:
 import numpy as np
import sounddevice as sd

# ✅ Convert Text to Frequencies
def text_to_sound(text):
    sample_rate = 44100
    duration = 2  # seconds
    frequency = 1000 + (ord(text[0]) % 100) * 10  # Map character to frequency
    t = np.linspace(0, duration, int(sample_rate * duration), False)
    sound_wave = 0.5 * np.sin(2 * np.pi * frequency * t)

    print("🔊 Playing encoded sound...")
    sd.play(sound_wave, samplerate=sample_rate)
    sd.wait()

text_to_sound("H")  # Send a character over sound
text_to_sound("e")  # Send a character over sound
text_to_sound("l")  # Send a character over sound


🔊 Playing encoded sound...
🔊 Playing encoded sound...
🔊 Playing encoded sound...


In [1]:
import numpy as np
import sounddevice as sd

# ✅ Parameters for FSK
SAMPLE_RATE = 44100  # Standard audio sampling rate
DURATION = 0.02  # Duration of each bit in seconds
FREQ_0 = 2000  # Frequency for bit 0
FREQ_1 = 4000  # Frequency for bit 1

# ✅ Convert text to binary
def text_to_binary(text):
    return ''.join(format(ord(char), '08b') for char in text)

# ✅ Generate FSK-modulated waveform
def generate_fsk_wave(binary_data):
    t = np.linspace(0, DURATION, int(SAMPLE_RATE * DURATION), False)
    waveform = np.concatenate([
        0.5 * np.sin(2 * np.pi * (FREQ_1 if bit == '1' else FREQ_0) * t)
        for bit in binary_data
    ])
    return waveform

# ✅ Play the sound
def text_to_fsk_sound(text):
    binary_data = text_to_binary(text)  # Convert text to binary
    print(f"🔠 Text: {text}")
    print(f"🖥 Binary: {binary_data}")

    waveform = generate_fsk_wave(binary_data)  # Generate sound wave
    print("🔊 Playing FSK-modulated sound...")
    
    sd.play(waveform, samplerate=SAMPLE_RATE)  # Play the sound
    sd.wait()
def decode_fsk_sound(waveform):
    binary_data = fsk_wave_to_binary(waveform)
    text = binary_to_text(binary_data)
    return text


# 🚀 Example Usage
if __name__ == "__main__":
    wave=text_to_fsk_sound("hello")  # Converts "Hi" to FSK sound
    decoded_text = decode_fsk_sound(wave)  # Decode the waveform back to text
    print(f"Decoded Text: {decoded_text}")

🔠 Text: hello
🖥 Binary: 0110100001100101011011000110110001101111
🔊 Playing FSK-modulated sound...


NameError: name 'fsk_wave_to_binary' is not defined

In [6]:
import numpy as np
import sounddevice as sd

# ✅ Parameters for FSK
SAMPLE_RATE = 44100  # Standard audio sampling rate
DURATION = 0.02  # Duration of each bit in seconds
FREQ_0 = 2000  # Frequency for bit 0
FREQ_1 = 4000  # Frequency for bit 1

# ✅ Convert text to binary
def text_to_binary(text):
    return ''.join(format(ord(char), '08b') for char in text)

# ✅ Generate FSK-modulated waveform
def generate_fsk_wave(binary_data):
    t = np.linspace(0, DURATION, int(SAMPLE_RATE * DURATION), False)
    waveform = np.concatenate([
        0.5 * np.sin(2 * np.pi * (FREQ_1 if bit == '1' else FREQ_0) * t)
        for bit in binary_data
    ])
    return waveform

# ✅ Play the sound and return the waveform
def text_to_fsk_sound(text):
    binary_data = text_to_binary(text)  # Convert text to binary
    print(f"🔠 Text: {text}")
    print(f"🖥 Binary: {binary_data}")

    waveform = generate_fsk_wave(binary_data)  # Generate sound wave
    print("🔊 Playing FSK-modulated sound...")
    
    sd.play(waveform, samplerate=SAMPLE_RATE)  # Play the sound
    sd.wait()
    
    return waveform  # Return the waveform

# ✅ Decode FSK waveform back to text
def fsk_wave_to_binary(waveform):
    # Placeholder for decoding logic
    # Implement the decoding logic here
    pass

def binary_to_text(binary_data):
    # Placeholder for binary-to-text conversion logic
    # Implement the conversion logic here
    pass

def decode_fsk_sound(waveform):
    binary_data = fsk_wave_to_binary(waveform)
    text = binary_to_text(binary_data)
    return text

# 🚀 Example Usage
if __name__ == "__main__":
    wave = text_to_fsk_sound("hello")  # Converts "hello" to FSK sound and returns the waveform
    # The following decoding logic will work once `fsk_wave_to_binary` and `binary_to_text` are implemented
    decoded_text = decode_fsk_sound(wave)  # Decode the waveform back to text
    print(f"Decoded Text: {decoded_text}")

🔠 Text: hello
🖥 Binary: 0110100001100101011011000110110001101111
🔊 Playing FSK-modulated sound...
Decoded Text: None


In [19]:
def text_to_fsk_sound(text):
    binary_data = text_to_binary(text)  # Convert text to binary
    print(f" Text: {text}") 
    print(f" Binary: {binary_data}") 

    waveform = generate_fsk_wave(binary_data)  # Generate sound wave
    print("FSK-modulated sound generated.") 
    sd.play(waveform, samplerate=SAMPLE_RATE)  # Play the sound
    sd.wait()  # Wait until sound is finished playing
    return waveform  # Return the generated waveform instead of playing it

In [ ]:
import numpy as np
from scipy.signal import hilbert

def fsk_wave_to_binary(waveform):
    samples_per_bit = int(SAMPLE_RATE * DURATION)  # Number of samples per bit
    binary_data = ""

    for i in range(0, len(waveform), samples_per_bit):
        segment = waveform[i:i+samples_per_bit]  # Extract bit-sized segment

        # Hilbert transform to extract envelope (frequency detection)
        analytic_signal = hilbert(segment)
        amplitude_envelope = np.abs(analytic_signal)
        
        # Determine dominant frequency using zero-crossing rate
        zero_crossings = np.where(np.diff(np.sign(segment)))[0]
        frequency = len(zero_crossings) / (2 * DURATION)  # Approximate frequency

        # Assign bit based on closest frequency
        if abs(frequency - FREQ_1) < abs(frequency - FREQ_0):
            binary_data += "1"
        else:
            binary_data += "0"

    return binary_data

# Convert binary back to text
def binary_to_text(binary_data):
    chars = [binary_data[i:i+8] for i in range(0, len(binary_data), 8)]
    return ''.join(chr(int(char, 2)) for char in chars if len(char) == 8)

#  Main decoding function
def decode_fsk_sound(waveform):
    binary_data = fsk_wave_to_binary(waveform)
    text = binary_to_text(binary_data)
    return text


if __name__ == "__main__":
    wave = text_to_fsk_sound("hello ")  # Converts "Hi" to FSK sound
    decoded_text = decode_fsk_sound(wave)  # Decode the waveform back to text
    print(f"Decoded Text: {decoded_text}")  # Print the decoded text

Decoded Text: hello 


In [11]:
import numpy as np
import sounddevice as sd
from scipy.signal import hilbert

# Parameters for FSK
SAMPLE_RATE = 44100  # Standard audio sampling rate
DURATION = 0.02  # Duration of each bit in seconds
FREQ_0 = 2000  # Frequency for bit 0
FREQ_1 = 4000  # Frequency for bit 1

# Convert text to binary
def text_to_binary(text):
    return ''.join(format(ord(char), '08b') for char in text)

# Generate FSK-modulated waveform
def generate_fsk_wave(binary_data):
    t = np.linspace(0, DURATION, int(SAMPLE_RATE * DURATION), False)
    waveform = np.concatenate([
        0.5 * np.sin(2 * np.pi * (FREQ_1 if bit == '1' else FREQ_0) * t)
        for bit in binary_data
    ])
    return waveform

# Play the sound and return the waveform
def text_to_fsk_sound(text):
    binary_data = text_to_binary(text)  # Convert text to binary
    print(f"🔠 Text: {text}")
    print(f"🖥 Binary: {binary_data}")

    waveform = generate_fsk_wave(binary_data)  # Generate sound wave
    print("🔊 Playing FSK-modulated sound...")
    
    sd.play(waveform, samplerate=SAMPLE_RATE)  # Play the sound
    sd.wait()
    
    return waveform  # Return the waveform

# Decode FSK waveform back to binary
def fsk_wave_to_binary(waveform):
    samples_per_bit = int(SAMPLE_RATE * DURATION)  # Number of samples per bit
    binary_data = ""

    for i in range(0, len(waveform), samples_per_bit):
        segment = waveform[i:i+samples_per_bit]  # Extract bit-sized segment

        # Hilbert transform to extract envelope (frequency detection)
        analytic_signal = hilbert(segment)
        amplitude_envelope = np.abs(analytic_signal)
        
        # Determine dominant frequency using zero-crossing rate
        zero_crossings = np.where(np.diff(np.sign(segment)))[0]
        frequency = len(zero_crossings) / (2 * DURATION)  # Approximate frequency

        # Assign bit based on closest frequency
        if abs(frequency - FREQ_1) < abs(frequency - FREQ_0):
            binary_data += "1"
        else:
            binary_data += "0"

    return binary_data

# Convert binary back to text
def binary_to_text(binary_data):
    chars = [binary_data[i:i+8] for i in range(0, len(binary_data), 8)]
    return ''.join(chr(int(char, 2)) for char in chars if len(char) == 8)

# Decode FSK waveform back to text
def decode_fsk_sound(waveform):
    binary_data = fsk_wave_to_binary(waveform)
    text = binary_to_text(binary_data)
    return text


if __name__ == "__main__":
    
    wave = text_to_fsk_sound("hello")  # Converts "hello" to FSK sound and returns the waveform
    decoded_text = decode_fsk_sound(wave)  # Decode the waveform back to text
    print(f"Decoded Text: {decoded_text}")

🔠 Text: hello
🖥 Binary: 0110100001100101011011000110110001101111
🔊 Playing FSK-modulated sound...
Decoded Text: hello


In [ ]:
#Main Code

import speech_recognition as sr
import numpy as np
import sounddevice as sd
from scipy.signal import hilbert

# Parameters for FSK
SAMPLE_RATE = 44100  # Standard audio sampling rate
DURATION = 0.02  # Duration of each bit in seconds
FREQ_0 = 2000  # Frequency for bit 0
FREQ_1 = 4000  # Frequency for bit 1

# Convert text to binary
def text_to_binary(text):
    return ''.join(format(ord(char), '08b') for char in text)

# Generate FSK-modulated waveform
def generate_fsk_wave(binary_data):
    t = np.linspace(0, DURATION, int(SAMPLE_RATE * DURATION), False)
    waveform = np.concatenate([
        0.5 * np.sin(2 * np.pi * (FREQ_1 if bit == '1' else FREQ_0) * t)
        for bit in binary_data
    ])
    return waveform

# Play the sound and return the waveform
def text_to_fsk_sound(text):
    binary_data = text_to_binary(text)  # Convert text to binary
    print(f"🔠 Text: {text}")
    print(f"🖥 Binary: {binary_data}")

    waveform = generate_fsk_wave(binary_data)  # Generate sound wave
    print("🔊 Playing FSK-modulated sound...")
    
    sd.play(waveform, samplerate=SAMPLE_RATE)  # Play the sound
    sd.wait()
    
    return waveform  

# Decode FSK waveform back to binary
def fsk_wave_to_binary(waveform):
    samples_per_bit = int(SAMPLE_RATE * DURATION)  # Number of samples per bit
    binary_data = ""

    for i in range(0, len(waveform), samples_per_bit):
        segment = waveform[i:i+samples_per_bit]  # Extract bit-sized segment

        # Hilbert transform to extract envelope (frequency detection)
        analytic_signal = hilbert(segment)
        amplitude_envelope = np.abs(analytic_signal)
        
        # Determine dominant frequency using zero-crossing rate
        zero_crossings = np.where(np.diff(np.sign(segment)))[0]
        frequency = len(zero_crossings) / (2 * DURATION)  # Approximate frequency

        # Assign bit based on closest frequency
        if abs(frequency - FREQ_1) < abs(frequency - FREQ_0):
            binary_data += "1"
        else:
            binary_data += "0"

    return binary_data

# Convert binary back to text
def binary_to_text(binary_data):
    chars = [binary_data[i:i+8] for i in range(0, len(binary_data), 8)]
    return ''.join(chr(int(char, 2)) for char in chars if len(char) == 8)

# Decode FSK waveform back to text
def decode_fsk_sound(waveform):
    binary_data = fsk_wave_to_binary(waveform)
    text = binary_to_text(binary_data)
    return text

# Speech recognition and FSK integration
def record_text():
    r = sr.Recognizer()
    try:
        with sr.Microphone() as source:
            print("🎤 Speak something...")
            r.adjust_for_ambient_noise(source, duration=0.2)
            audio = r.listen(source)
            recognized_text = r.recognize_google(audio)
            print(f"📝 Recognized Text: {recognized_text}")
            return recognized_text
    except sr.RequestError as e:
        print(f"Could not request results; {e}")
    except sr.UnknownValueError:
        print("Speech recognition could not understand the audio.")
    return None

if __name__ == "__main__":
    # Step 1: Record text using speech recognition
    text = record_text()
    if text:
        # Step 2: Convert text to FSK-modulated sound
        waveform = text_to_fsk_sound(text)
        
        # Step 3: Decode the FSK-modulated sound back to text
        decoded_text = decode_fsk_sound(waveform)
        print(f"✅ Decoded Text: {decoded_text}")

🎤 Speak something...
📝 Recognized Text: hello hello hello
🔠 Text: hello hello hello
🖥 Binary: 0110100001100101011011000110110001101111001000000110100001100101011011000110110001101111001000000110100001100101011011000110110001101111
🔊 Playing FSK-modulated sound...
✅ Decoded Text: hello hello hello
